In [1]:
import CGA_for_ttp as cga
import numpy as np
from obj_func import obj_func
from read_instance import read_instance
import random
import seaborn as sns
from random import randint
import sys 
from sklearn.cluster import KMeans
import math
from python_tsp.heuristics import solve_tsp_simulated_annealing
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt
from pylab import savefig
import os
import csv
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning) 

In [2]:
#file=open(r"E:\monit\Documents\TTP\Instancias\a280-ttp\a280_n279_bounded-strongly-corr_01.ttp","r")
file=open(r"E:\monit\Documents\TTP\Instancias\eil51-ttp\eil51_n250_uncorr_01.ttp","r")
#file=open(r"E:\monit\Documents\TTP\Instancias\a280-ttp\my_test.ttp","r")
#dic=read_instance(file)


In [3]:
def create_knap_pop(dic,y,x):
    pop_knap=np.zeros((y,x,dic["number of items"]))
    for i in range(pop_knap.shape[0]):
        for l in range(pop_knap.shape[1]):
            item_pick=[]
            for x in range(dic["number of items"]):
                item_pick.append(random.randint(0, 1))
            item_pick2=np.array(item_pick)
            pop_knap[i,l]=item_pick2
    return pop_knap

In [4]:
def create_tsp_pop(dic,y,x):
    pop_tsp=np.zeros((y,x,dic["number cities"]-1))
    for i in range(pop_tsp.shape[0]):
        for l in range(pop_tsp.shape[1]):
            order_cities=list(range(2,dic["number cities"]+1 ))
            random.shuffle(order_cities)
            pop_tsp[i,l,:]=order_cities
    return pop_tsp

In [5]:
def fit_loop(dic,pop_tsp,pop_knap):
    Y=pop_tsp.shape[0]
    X=pop_tsp.shape[1]
    fitness=np.zeros((Y,X))
    for i in range(Y):
        for l in range(X):
            fitness[i,l]=obj_func(dic,pop_tsp[i,l,:],pop_knap[i,l,:])
    return fitness

In [6]:
def k_fix(dic,pop_tsp):
    coords=dic['coords citites'] 
    X=pop_tsp.shape[0]
    Y=pop_tsp.shape[1]
    Z=pop_tsp.shape[2]
    pop_tsp=np.reshape(pop_tsp,(X*Y,pop_tsp.shape[2]))
    L=int(math.sqrt(dic['number cities']-1)+.5)
    pop_cords=np.zeros((Z,2))
    randomlist = random.sample(range(0,X*Y), int(X*Y*.10))
    #crear los la poblacion para hacer los clusters
    for n in randomlist:
        for x in range(Z):
            pre_cords=coords[int(pop_tsp[n,x])-1]
            pop_cords[x]=[int(float(pre_cords[0])),int(float(pre_cords[1]))]
        #crear los clusters
        kmeans = KMeans(n_clusters=L, random_state=0).fit(pop_cords)
        #print(kmeans.labels_)
        centers=kmeans.cluster_centers_
        #print(centers)
        #separar los clusters 
        clusters=[]

        for l in range(L):
            clu_pre=[]
            for i in range(Z):
                if kmeans.labels_[i]==l:
                    clu_pre.append(i)
            clusters.append(clu_pre)
        #print(clusters)
        #clusters fixedd by tsp simulated anneling
        clusters_fixed=[]
        for e in clusters:
            test=e
            dist_test=[]
            for o in range(len(test)):
                 dist_test.append((int(coords[test[o]-1][0]),int(coords[test[o]-1][1])))
            clusters_fixed.append(mini_tsp(test,dist_test))

        dist_cluster=[]

        for a in clusters_fixed:
            pre_clus=a
            pre_dist=[]
            for p in pre_clus:
                #print(p)
                pre_cords2=coords[p-1]
                pop_cords2=[int(pre_cords2[0]),int(pre_cords2[1])]
                #print(pop_cords2)
                pre_dist.append(pop_cords2)
            dist_cluster.append(pre_dist)

        #print(clusters_fixed,dist_cluster)        
        #print(clusters_fixed)
        #centroid distance search
        merge_order=[]
        centers2=centers.copy()
        #clusters_merge=np.array(clusters_fixed.copy())
        clusters_merge=np.array(clusters_fixed)
        for x in range(L-1):
            centroids_matrix=distance_matrix(centers2,centers2)
            centroids_matrix=np.where(centroids_matrix == 0, np.Inf, centroids_matrix)
            #print(centroids_matrix,"centroids")
            minx=list(zip(*np.where(centroids_matrix==np.amin(centroids_matrix))))
            #print(minx)
            merge_order.append(minx[0])
            new_centroid=(centers[minx[0][0]]+centers[minx[0][1]]) /2
            #print(new_centroid,"new centro")
            centers2[minx[0][0]]=new_centroid
            centers2=np.delete(centers2,minx[0][1],0)
            #print(minx)
            cluster1=clusters_merge[minx[0][0]]
            cluster2=clusters_merge[minx[0][1]]
            cluster_dist1=dist_cluster[minx[0][0]]
            cluster_dist2=dist_cluster[minx[0][1]]
            cluster_matrix=distance_matrix(cluster_dist1,cluster_dist2)
            cluster_matrix=np.where(cluster_matrix == 0, np.Inf, cluster_matrix)
            minc=list(zip(*np.where(cluster_matrix==np.amin(cluster_matrix))))
            cluster1=np.roll(cluster1,-minc[0][0])
            cluster2=np.roll(cluster2,-minc[0][1]-1)
            new_clust=np.append(cluster2,cluster1)

            clusters_merge[minx[0][0]]=new_clust
            clusters_merge=np.delete(clusters_merge, minx[0][1],0)


        pop_tsp[n,:]=clusters_merge[0]

    pop_tsp=np.reshape(pop_tsp,(X,Y,Z))
    return(pop_tsp)


In [18]:
from python_tsp.heuristics import solve_tsp_local_search
def mini_tsp(tsp,coords):
    #distance_matrixe=np.zeros((len(tsp),len(tsp)))
    '''
    for x in range (len(coords)):
        for i in range (len(coords)):
            distance_matrixe[x,i]=np.linalg.norm([(coords[x][0]-coords[i][0]),(coords[x][1]-coords[i][1])])
            '''
    dist=distance_matrix(coords,coords)
    #print(dist)
    permutation, distance = solve_tsp_local_search(dist)
    #print(permutation,distance)  
    fix=[]
    for i in permutation:
        fix.append(tsp[i])
    return(fix)

In [20]:
def iter(X,Y,mr,dic):
    pop_knap=create_knap_pop(dic,Y,X)
    pop_tsp=create_tsp_pop(dic,Y,X)
    pop_tsp=k_fix(dic,pop_tsp)
    fitness=fit_loop(dic,pop_tsp,pop_knap)
    #iterations=1000
    l=0
    z=True
    count=0
    lastfit=np.amax(fitness)
    #result=[]
    #result.append(lastfit)
    while z:
        pop_knap,pop_tsp=cga.crossover(fitness,pop_knap,pop_tsp,Y,X,mr) 
        if l%100==0:
            print("generation: ",l, " maximo: ", np.amax(fitness))
        l=l+1
        #if l%100==0:
          #pop_tsp=k_fix(dic,pop_tsp)

        fitness=fit_loop(dic,pop_tsp,pop_knap)
        #result.append(np.amax(fitness))  

        if lastfit>=np.amax(fitness):
            count=count+1
        else:
            count=0
        lastfit=np.amax(fitness)
        if count==100 :
            z=False


    #p1 = sns.heatmap(fitness)

    return lastfit,l
  #print(maximos)

  #p1 = sns.heatmap(fitness)  
   

In [21]:
directory=r"E:\monit\Documents\TTP\Instancias\bier127-ttp"
results=[]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        file=open(f,"r")
    nombre=f[44:]
    print('currently doing: ',nombre )
    dic=read_instance(file)
    res,gen=iter(20,10,.1,dic)
    print((nombre,gen,res))
    results.append((nombre,gen,res))


currently doing:  p\bier127_n1260_bounded-strongly-corr_01.ttp
generation:  0  maximo:  -1971964.1118879775
generation:  100  maximo:  -1961522.3036691917
generation:  200  maximo:  -1691310.1650434441
generation:  300  maximo:  -1525521.3966336944
generation:  400  maximo:  -1364239.9705165068
generation:  500  maximo:  -1275726.3003539722
generation:  600  maximo:  -1128147.5266594987
generation:  700  maximo:  -1058252.8394536788
generation:  800  maximo:  -991683.2085196753
generation:  900  maximo:  -928853.4989729032
generation:  1000  maximo:  -847031.6184304599
generation:  1100  maximo:  -783735.5801306317
generation:  1200  maximo:  -724263.2439501024
generation:  1300  maximo:  -674415.0847252964
generation:  1400  maximo:  -620420.3250192003
generation:  1500  maximo:  -549814.7964889708
generation:  1600  maximo:  -441306.10880815645
generation:  1700  maximo:  -286160.0353248224
generation:  1800  maximo:  -165183.19341590785
generation:  1900  maximo:  -55910.58827429992

generation:  3700  maximo:  329245.3579724202
generation:  3800  maximo:  331331.2003666603
generation:  3900  maximo:  332507.9091615534
generation:  4000  maximo:  334280.4188598921
generation:  4100  maximo:  339405.9720372005
generation:  4200  maximo:  340989.87712317513
generation:  4300  maximo:  341952.3913524515
generation:  4400  maximo:  343964.56647124013
generation:  4500  maximo:  345148.61730834073
generation:  4600  maximo:  346929.17938381486
generation:  4700  maximo:  348058.42725147406
generation:  4800  maximo:  348756.23379809415
generation:  4900  maximo:  349094.68010654085
generation:  5000  maximo:  351446.94704502507
generation:  5100  maximo:  352037.7725058612
generation:  5200  maximo:  354771.98247096495
generation:  5300  maximo:  355095.4864898105
generation:  5400  maximo:  355461.72354674735
generation:  5500  maximo:  355765.8648658903
generation:  5600  maximo:  356055.3707889876
generation:  5700  maximo:  356321.321993244
generation:  5800  maximo

generation:  5300  maximo:  18992.818466272787
generation:  5400  maximo:  21584.932362724794
generation:  5500  maximo:  23020.24297003995
generation:  5600  maximo:  24277.567684254493
generation:  5700  maximo:  29319.912162719877
generation:  5800  maximo:  30401.590766353533
generation:  5900  maximo:  31885.617307948763
generation:  6000  maximo:  33267.788031361764
generation:  6100  maximo:  33877.449086314766
generation:  6200  maximo:  39304.22160306224
generation:  6300  maximo:  40894.01841353625
generation:  6400  maximo:  43762.50086774875
generation:  6500  maximo:  44680.77007041767
generation:  6600  maximo:  45510.86542495107
generation:  6700  maximo:  46047.02298560832
generation:  6800  maximo:  46579.01946701715
generation:  6900  maximo:  47385.458355405135
generation:  7000  maximo:  47787.4761509879
generation:  7100  maximo:  48413.45919592865
generation:  7200  maximo:  48911.31793706678
generation:  7300  maximo:  49319.951457712334
generation:  7400  maximo

KeyboardInterrupt: 

In [ ]:
with open('E:\monit\Documents\TTP\Results\berlin52-ttp,150pop,mr=.1'+'.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Name", 'Generations','Max value'])
    for x in results:
        writer.writerow([x[0], x[1],x[2]])